## 날짜형 column, 주소 columns split

In [36]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re
fe = fm.FontEntry(fname=r'font/NanumGothic.otf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
data_path = '../data/'

data = pd.read_csv(f'{data_path}240717_ing_not_null.csv')

In [3]:
df = data.copy()

In [4]:
df.isna().sum()

전용면적(㎡)                           0
층                                 0
건축년도                              0
target                         9272
is_test                           0
계약년월일                             0
k-전용면적별세대현황(60㎡이하)                0
k-전용면적별세대현황(60㎡~85㎡이하)            0
k-85㎡~135㎡이하                      0
k-135㎡초과                          0
주차대수                              0
Nearest_Bus_Station_Name          0
Nearest_Subway_Station_Name       0
5층 동수                             0
5층 세대수                            0
6~10층 동수                          0
6~10층 세대수                         0
11~15층 동수                         0
11~15층 세대수                        0
16~20층 동수                         0
16~20층 세대수                        0
21층이상 동수                          0
21층이상 세대수                         0
시점                                0
국고채                               0
회사채                               0
국고채-회사채                           0
생산자물가지수                     

In [5]:
df['계약년월일'] = pd.to_datetime(df['계약년월일'])

In [6]:
df['계약_년도'] = df['계약년월일'].dt.year
df['계약_월'] = df['계약년월일'].dt.month
df['계약_일'] = df['계약년월일'].dt.day

In [7]:
df.drop(columns='계약년월일', inplace=True)

In [8]:
df[df['층']=='0']
df[df['전용면적(㎡)']<=10]
df['층'].max() #69
df['층'].min() # -4

-4

In [9]:
df.head(1)

,전용면적(㎡),층,건축년도,target,is_test,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,Nearest_Bus_Station_Name,Nearest_Subway_Station_Name,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,시점,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,좌표X_jm,좌표Y_jm,번지주소_2,도로명_fill,최단거리초등학교명,최단거리초등학교Km,좌표X_2,좌표Y_2,1번째_가까운_역_이름,1번째_가까운_역_호선,1번째_가까운_역_거리,1번째_가까운_역_도보시간,2번째_가까운_역_이름,2번째_가까운_역_호선,2번째_가까운_역_거리,2번째_가까운_역_도보시간,3번째_가까운_역_이름,3번째_가까운_역_호선,3번째_가까운_역_거리,3번째_가까운_역_도보시간,5분이하_역_개수,5분초과_10분이하_역_개수,10분초과_15분이하_역_개수,15분초과_20분이하_역_개수,건물명,세대수_수정,계약_년도,계약_월,계약_일
0,79.97,3,1987,124000.0,0,20.0,250.0,0.0,0.0,262.0,개포우성아파트,구룡,8.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201712,2.1,2.649,0.549,99.09,97.698,13740.0,8484.0,87.701,93.042,127.056859,37.476276,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3,서울포이초등학교,0.385832,127.056859,37.476276,구룡,분당선,1187.672025,17.81508,매봉,3호선,1483.43809,22.251571,도곡,3호선,1633.265245,24.498979,0,0,0,1,개포6차우성,270.0,2017,12,8


In [10]:
float_columns = df.select_dtypes(include=['float']).columns
float_columns

Index(['전용면적(㎡)', 'target', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', '주차대수', '5층 동수', '5층 세대수', '6~10층 동수',
       '6~10층 세대수', '11~15층 동수', '11~15층 세대수', '16~20층 동수', '16~20층 세대수',
       '21층이상 동수', '21층이상 세대수', '국고채', '회사채', '국고채-회사채', '생산자물가지수', '소비자물가지수',
       '서울특별시_주택매매거래량', '서울특별시_아파트매매거래량', '전국매매가격지수', '전국전세가격지수', '좌표X_jm',
       '좌표Y_jm', '최단거리초등학교Km', '좌표X_2', '좌표Y_2', '1번째_가까운_역_거리',
       '1번째_가까운_역_도보시간', '2번째_가까운_역_거리', '2번째_가까운_역_도보시간', '3번째_가까운_역_거리',
       '3번째_가까운_역_도보시간', '세대수_수정'],
      dtype='object')

In [11]:
not_float = ['k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', '주차대수', '5층 동수', '5층 세대수', '6~10층 동수',
       '6~10층 세대수', '11~15층 동수', '11~15층 세대수', '16~20층 동수', '16~20층 세대수',
       '21층이상 동수', '21층이상 세대수', '세대수_수정']

In [12]:
df[not_float] = df[not_float].astype('int')
df.head(1)


,전용면적(㎡),층,건축년도,target,is_test,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,Nearest_Bus_Station_Name,Nearest_Subway_Station_Name,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,시점,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,좌표X_jm,좌표Y_jm,번지주소_2,도로명_fill,최단거리초등학교명,최단거리초등학교Km,좌표X_2,좌표Y_2,1번째_가까운_역_이름,1번째_가까운_역_호선,1번째_가까운_역_거리,1번째_가까운_역_도보시간,2번째_가까운_역_이름,2번째_가까운_역_호선,2번째_가까운_역_거리,2번째_가까운_역_도보시간,3번째_가까운_역_이름,3번째_가까운_역_호선,3번째_가까운_역_거리,3번째_가까운_역_도보시간,5분이하_역_개수,5분초과_10분이하_역_개수,10분초과_15분이하_역_개수,15분초과_20분이하_역_개수,건물명,세대수_수정,계약_년도,계약_월,계약_일
0,79.97,3,1987,124000.0,0,20,250,0,0,262,개포우성아파트,구룡,8,270,0,0,0,0,0,0,0,0,201712,2.1,2.649,0.549,99.09,97.698,13740.0,8484.0,87.701,93.042,127.056859,37.476276,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3,서울포이초등학교,0.385832,127.056859,37.476276,구룡,분당선,1187.672025,17.81508,매봉,3호선,1483.43809,22.251571,도곡,3호선,1633.265245,24.498979,0,0,0,1,개포6차우성,270,2017,12,8


In [13]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 61 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   전용면적(㎡)                      1128094 non-null  float64
 1   층                            1128094 non-null  int64  
 2   건축년도                         1128094 non-null  int64  
 3   target                       1118822 non-null  float64
 4   is_test                      1128094 non-null  int64  
 5   k-전용면적별세대현황(60㎡이하)           1128094 non-null  int32  
 6   k-전용면적별세대현황(60㎡~85㎡이하)       1128094 non-null  int32  
 7   k-85㎡~135㎡이하                 1128094 non-null  int32  
 8   k-135㎡초과                     1128094 non-null  int32  
 9   주차대수                         1128094 non-null  int32  
 10  Nearest_Bus_Station_Name     1128094 non-null  object 
 11  Nearest_Subway_Station_Name  1128094 non-null  object 
 12  5층 동수                        1128094 non-n

In [14]:
df.isna().sum()[50:]

3번째_가까운_역_거리        0
3번째_가까운_역_도보시간      0
5분이하_역_개수           0
5분초과_10분이하_역_개수     0
10분초과_15분이하_역_개수    0
15분초과_20분이하_역_개수    0
건물명                 0
세대수_수정              0
계약_년도               0
계약_월                0
계약_일                0
dtype: int64

In [15]:
test_df = df[['번지주소_2', '도로명_fill']]
test_df.head(3)

,번지주소_2,도로명_fill
0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
1,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3


In [16]:
test_df[['시', '구','동']] = test_df['번지주소_2'].str.split(' ', n=2, expand=True)

In [17]:
test_df[['동', '번지']] = test_df['동'].str.split(' ', n=1, expand=True)

In [18]:
test_df[['시', '구','도로명']] = test_df['도로명_fill'].str.split(' ', n=2, expand=True)

In [19]:
test_df.head(1)

,번지주소_2,도로명_fill,시,구,동,번지,도로명
0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3,서울특별시,강남구,개포동,658-1,언주로 3


In [20]:
test_df.isna().sum()

번지주소_2      0
도로명_fill    0
시           0
구           0
동           0
번지          0
도로명         0
dtype: int64

In [22]:
df[['시', '구', '동', '번지', '도로명']] = test_df[['시', '구', '동', '번지', '도로명']].copy()

In [23]:
df.head(1)

,전용면적(㎡),층,건축년도,target,is_test,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,Nearest_Bus_Station_Name,Nearest_Subway_Station_Name,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,시점,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,좌표X_jm,좌표Y_jm,번지주소_2,도로명_fill,최단거리초등학교명,최단거리초등학교Km,좌표X_2,좌표Y_2,1번째_가까운_역_이름,1번째_가까운_역_호선,1번째_가까운_역_거리,1번째_가까운_역_도보시간,2번째_가까운_역_이름,2번째_가까운_역_호선,2번째_가까운_역_거리,2번째_가까운_역_도보시간,3번째_가까운_역_이름,3번째_가까운_역_호선,3번째_가까운_역_거리,3번째_가까운_역_도보시간,5분이하_역_개수,5분초과_10분이하_역_개수,10분초과_15분이하_역_개수,15분초과_20분이하_역_개수,건물명,세대수_수정,계약_년도,계약_월,계약_일,시,구,동,번지,도로명
0,79.97,3,1987,124000.0,0,20,250,0,0,262,개포우성아파트,구룡,8,270,0,0,0,0,0,0,0,0,201712,2.1,2.649,0.549,99.09,97.698,13740.0,8484.0,87.701,93.042,127.056859,37.476276,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3,서울포이초등학교,0.385832,127.056859,37.476276,구룡,분당선,1187.672025,17.81508,매봉,3호선,1483.43809,22.251571,도곡,3호선,1633.265245,24.498979,0,0,0,1,개포6차우성,270,2017,12,8,서울특별시,강남구,개포동,658-1,언주로 3


In [28]:
#df.to_csv(f'{data_path}240717_ing_not_null_2.csv', index=False)

In [24]:
# del test_df

In [29]:
df_columns = ['계약_년도',
'계약_월',
'계약_일',
'건축년도',
'시',
'구',
'동',
'번지',
'도로명',
'건물명',
'층',

'전용면적(㎡)',
'k-전용면적별세대현황(60㎡이하)',
'k-전용면적별세대현황(60㎡~85㎡이하)',
'k-85㎡~135㎡이하',
'k-135㎡초과',
'주차대수',

'5층 동수',
'5층 세대수',
'6~10층 동수',
'6~10층 세대수',
'11~15층 동수',
'11~15층 세대수',
'16~20층 동수',
'16~20층 세대수',
'21층이상 동수',
'21층이상 세대수',
'세대수_수정',

# '시점',
# '국고채',
# '회사채',
# '국고채-회사채',
# '생산자물가지수',
# '소비자물가지수',
# '서울특별시_주택매매거래량',
# '서울특별시_아파트매매거래량',
# '전국매매가격지수',
# '전국전세가격지수',

'좌표X_jm',
'좌표Y_jm',

'최단거리초등학교명',
'최단거리초등학교Km',

'1번째_가까운_역_이름',
'1번째_가까운_역_호선',
'1번째_가까운_역_거리',
'1번째_가까운_역_도보시간',
'2번째_가까운_역_이름',
'2번째_가까운_역_호선',
'2번째_가까운_역_거리',
'2번째_가까운_역_도보시간',
'3번째_가까운_역_이름',
'3번째_가까운_역_호선',
'3번째_가까운_역_거리',
'3번째_가까운_역_도보시간',
'5분이하_역_개수',
'5분초과_10분이하_역_개수',
'10분초과_15분이하_역_개수',
'15분초과_20분이하_역_개수',

'target',
'is_test'
]

In [33]:
# del df

In [32]:
dataset = df[df_columns].copy()
dataset.head(2)

,계약_년도,계약_월,계약_일,건축년도,시,구,동,번지,도로명,건물명,층,전용면적(㎡),k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,세대수_수정,좌표X_jm,좌표Y_jm,최단거리초등학교명,최단거리초등학교Km,1번째_가까운_역_이름,1번째_가까운_역_호선,1번째_가까운_역_거리,1번째_가까운_역_도보시간,2번째_가까운_역_이름,2번째_가까운_역_호선,2번째_가까운_역_거리,2번째_가까운_역_도보시간,3번째_가까운_역_이름,3번째_가까운_역_호선,3번째_가까운_역_거리,3번째_가까운_역_도보시간,5분이하_역_개수,5분초과_10분이하_역_개수,10분초과_15분이하_역_개수,15분초과_20분이하_역_개수,target,is_test
0,2017,12,8,1987,서울특별시,강남구,개포동,658-1,언주로 3,개포6차우성,3,79.97,20,250,0,0,262,8,270,0,0,0,0,0,0,0,0,270,127.056859,37.476276,서울포이초등학교,0.385832,구룡,분당선,1187.672025,17.81508,매봉,3호선,1483.43809,22.251571,도곡,3호선,1633.265245,24.498979,0,0,0,1,124000.0,0
1,2017,12,22,1987,서울특별시,강남구,개포동,658-1,언주로 3,개포6차우성,4,79.97,20,250,0,0,262,8,270,0,0,0,0,0,0,0,0,270,127.056859,37.476276,서울포이초등학교,0.385832,구룡,분당선,1187.672025,17.81508,매봉,3호선,1483.43809,22.251571,도곡,3호선,1633.265245,24.498979,0,0,0,1,123500.0,0


In [34]:
object_col = dataset.select_dtypes(include=['object']).columns
object_col

Index(['시', '구', '동', '번지', '도로명', '건물명', '최단거리초등학교명', '1번째_가까운_역_이름',
       '1번째_가까운_역_호선', '2번째_가까운_역_이름', '2번째_가까운_역_호선', '3번째_가까운_역_이름',
       '3번째_가까운_역_호선'],
      dtype='object')

In [35]:
num_col = dataset.select_dtypes(include=['int64', 'float64']).columns
num_col

Index(['건축년도', '층', '전용면적(㎡)', '좌표X_jm', '좌표Y_jm', '최단거리초등학교Km',
       '1번째_가까운_역_거리', '1번째_가까운_역_도보시간', '2번째_가까운_역_거리', '2번째_가까운_역_도보시간',
       '3번째_가까운_역_거리', '3번째_가까운_역_도보시간', '5분이하_역_개수', '5분초과_10분이하_역_개수',
       '10분초과_15분이하_역_개수', '15분초과_20분이하_역_개수', 'target', 'is_test'],
      dtype='object')

In [37]:
dataset.to_csv(f'{data_path}240717_ing_not_null_3.csv', index=False)